# Solving a Murder Mystery using SQL

## The Prompt:

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a murder that occurred sometime on Jan. 15, 2018 and that it took place in SQL City. All the clues to this mystery are buried in a huge database, and you need to use SQL to navigate through this vast network of information. Your first step to solving the mystery is to retrieve the corresponding crime scene report from the police department's database.

## The Project:

The project prompt and the data has been collected from <a href="https://github.com/NUKnightLab/sql-mysteries">NUKnightLab's GitHub Repository</a>.

## \*\*USE murder Database\*\*

In [2]:
USE murder;
GO

Commands completed successfully.

Total execution time: 00:00:00.016

## Crime Scene Report:

It has been said that the crime happened on 15th January 2018 in SQL City and that it was a murder. Let's look into the crime scene report to start the investigation.

In [4]:
SELECT * FROM crime_scene_report 
WHERE date = 20180115 AND type = 'murder' AND city = 'SQL City';

(1 row affected)

Total execution time: 00:00:00.036

date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


### **The Report:**

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

According to the report, there were two witnesses:

- The first witness lives at the last house on Northwestern Dr
- The second witness, named Annabel, lives somewhere on Franklin Ave

Let's find the interview of these witnesses.

## Find the Witnesses:

Let's look for both the witnesses in the \`person\` table. 

The first one lives at the last house on Northwestern Drive.

In [6]:
SELECT TOP(1) * FROM person 
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC;

(1 row affected)

Total execution time: 00:00:00.028

id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


So, the first witness is Morty Schapiro. Now, let's look for the other witness whose name is Annabel and lives somewhere on Franklin Ave.

In [7]:
SELECT * FROM person 
WHERE name LIKE '%Annabel%' AND address_street_name = 'Franklin Ave';

(1 row affected)

Total execution time: 00:00:00.044

id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


We have also found the the second witness named Annabel Miller.

## Witness Interviews:

Now that we know who our two witnesses are, let's look into their interviews to find clues about the culprit. 

  

Let's start with Morty Schapiro's report.

In [11]:
SELECT * FROM interview
WHERE person_id IN (14887, 16371);

(2 rows affected)

Total execution time: 00:00:00.015

person_id,transcript
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


### **The Interview Transcript:**

- **Witness 1:** 'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'
- **Witness 2:** 'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

Both the witnesses seem to have seen the same person. Some key information we can gather from the interviews are:

- The killer is a male
- The killer is a member of 'Get Fit Now' gym
- Killer's membership number started with '48Z'
- Killer is a gold member
- Killer's car plate has the number 'H42W'
- Killer checked into the gym on January 9th 2018

## Search for Gym Member:

Since we know the killer is a 'Get Fit Now' gold member with the membership no. starting with '48Z', let's begin our search for the suspect from there.

In [12]:
SELECT * FROM get_fit_now_member 
WHERE id LIKE '48Z%' AND membership_status = 'gold';

(2 rows affected)

Total execution time: 00:00:00.043

id,person_id,name,membership_start_date,membership_status
48Z7A,28819,Joe Germuska,20160305,gold
48Z55,67318,Jeremy Bowers,20160101,gold


According to the witness description about the gym member, we have two suspects: Joe Germuska and Jeremy Bowers.

## Cross Check with Licence Plate:

We also know that the killer, who was a male, has a car containing the licence plate number 'H42W'. Let's cross check that information with the information received from the gym member data.

In [14]:
SELECT
    p.id AS person_id,
    name,
    dl.id AS license_id,
    gender,
    plate_number
FROM drivers_license dl 
INNER JOIN person p 
ON dl.id = p.license_id 
WHERE dl.gender = 'male' AND dl.plate_number LIKE '%H42W%';

(2 rows affected)

Total execution time: 00:00:00.031

person_id,name,license_id,gender,plate_number
51739,Tushar Chandra,664760,male,4H42WR
67318,Jeremy Bowers,423327,male,0H42W2


We again have two suspects who were male and had car with the plate number containing 'H42W'. Among these two, Jeremy Bowers is also a suspect based on our previous search. Hence, he is our prime suspect for now.

## Search Gym Check In Data:

Finally, according to the second witness of the case, the killer checked in at the 'Get Fit Now' gym on January 9th 2018 before Annabel Miller as she saw him working out while she was there. Let's use this information and find out who checked in on January 9th 2018 before Annabel and checked out after she checked in.

In [19]:
WITH annabel_check_in AS (
    SELECT * FROM get_fit_now_check_in
    WHERE membership_id = (
        SELECT id FROM get_fit_now_member 
        WHERE person_id = 16371
    )
    AND check_in_date = 20180109
)

SELECT gfm.name, gfm.person_id, gfc.membership_id, gfc.check_in_date, gfc.check_in_time, gfc.check_out_time
FROM get_fit_now_check_in gfc
INNER JOIN get_fit_now_member gfm
ON gfc.membership_id = gfm.id 
INNER JOIN annabel_check_in ac 
ON gfc.check_in_date = ac.check_in_date 
WHERE gfc.check_in_time <= ac.check_in_time AND gfc.check_out_time >= ac.check_in_time;

(3 rows affected)

Total execution time: 00:00:00.036

name,person_id,membership_id,check_in_date,check_in_time,check_out_time
Joe Germuska,28819,48Z7A,20180109,1600,1730
Jeremy Bowers,67318,48Z55,20180109,1530,1700
Annabel Miller,16371,90081,20180109,1600,1700
